In [1]:
import os
import csv
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

In [256]:
df = pd.read_csv('submission_20_original.csv')   # the latest is 20
df_sample = pd.read_csv('sample_submission.csv')[['id']]

df_1 = pd.read_csv('labels_DELF1_0_15000.csv')
df_2 = pd.read_csv('labels_DELF1_15000_30000.csv')
df_3 = pd.read_csv('labels_DELF1_30000_45000.csv')
df_4 = pd.read_csv('labels_DELF1_45000_60000.csv')
df_5 = pd.read_csv('labels_DELF1_60000_75000.csv')
df_6 = pd.read_csv('labels_DELF1_75000_90000.csv')
df_7 = pd.read_csv('labels_DELF1_90000_100000.csv')
df_8 = pd.read_csv('labels_DELF1_100000_110000.csv')
df_9 = pd.read_csv('labels_DELF1_110000_120000.csv')
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9] 

df_labels = pd.concat(frames, axis = 0).reset_index()
df_labels.shape

(117703, 2)

In [257]:
df_sample.shape

(117703, 1)

In [258]:
# result of DELF filtering I

censor = 180   # for submission 20

drop_idx = df_labels[df_labels['0'] <= censor].index
df = df.drop(drop_idx, axis = 0)
df.shape

(65960, 2)

In [259]:
df.head()

,id,landmarks
0,000088da12d664db,14255 0.26008448004722595
4,000270c9100de789,4404 0.4304988384246826
5,0002b0fab5d3ccc4,11491 0.546776294708252
9,0005456a82264bc8,4404 0.30132150650024414
10,00055cf2bfb5594a,10184 0.597339928150177


In [260]:
drop_idx.shape

(51743,)

In [261]:
df[df['landmarks'].isnull()]   # check

,id,landmarks


In [262]:
threshold = 0.4  # 0.4 for submission 20

df_tmp = df.dropna(how = 'any')
df_tmp['landmark_id'] = df_tmp['landmarks'].apply(lambda x: int(str(x).split(' ')[0]))
df_tmp['confidence'] = df_tmp['landmarks'].apply(lambda x: float(str(x).split(' ')[-1]))
df_tmp[['confidence']].describe()

df_tmp = df_tmp[df_tmp['confidence'] > threshold]
#df_tmp = df_tmp[np.logical_and(df_tmp['confidence'] > threshold, df_tmp['confidence'] < threshold * 5)]
df_tmp = df_tmp.drop(['landmark_id', 'confidence'], axis = 1)
print(df_tmp.shape)

(40283, 2)


In [263]:
df_tmp.head()

,id,landmarks
4,000270c9100de789,4404 0.4304988384246826
5,0002b0fab5d3ccc4,11491 0.546776294708252
10,00055cf2bfb5594a,10184 0.597339928150177
12,0006aea5b6f4eaaa,6603 0.9498259425163269
13,0006bbfa00dd6c0f,9779 0.5040563344955444


In [264]:
df = df_tmp

In [265]:
# result of DELF filtering II

test_revised = pd.read_csv('test_revised_based_on_20.csv')
test_revised = test_revised.sort_values(by = 'confidence', ascending = False)
test_revised = test_revised.sort_values(by = 'landmark_id', ascending = True)
test_revised = test_revised.reset_index().drop('index', axis = 1)
test_revised.shape

(40283, 4)

In [266]:
df_1 = pd.read_csv('labels_DELF2_sub20_0_2500.csv')
df_2 = pd.read_csv('labels_DELF2_sub20_2500_5000.csv')
df_3 = pd.read_csv('labels_DELF2_sub20_5000_7500.csv')
df_4 = pd.read_csv('labels_DELF2_sub20_7500_9700.csv')
df_5 = pd.read_csv('labels_DELF2_sub20_9700_11170.csv')
df_6 = pd.read_csv('labels_DELF2_sub20_11170_11600.csv')
df_7 = pd.read_csv('labels_DELF2_sub20_11600_12000.csv')
df_8 = pd.read_csv('labels_DELF2_sub20_12000_14100.csv')
df_9 = pd.read_csv('labels_DELF2_sub20_14100_14951.csv')

frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9] 
df_labels_2 = pd.concat(frames, axis = 0).reset_index().drop('index', axis = 1)
df_labels_2.shape

(40283, 6)

In [268]:
df_labels_2.head()

,0,1,2,3,4,5
0,22,14.75,14,11.25,8,7.25
1,21,14.75,17,11.50,9,6.75
2,38,26.00,20,14.25,11,8.00
3,52,37.50,27,17.25,19,11.00
4,82,66.00,19,16.00,18,14.00


In [269]:
test_revised = pd.concat([test_revised, df_labels_2], axis = 1).drop('landmarks', axis = 1)
test_revised.head()

,id,landmark_id,confidence,0,1,2,3,4,5
0,9dcd24b41ff39a0e,0,0.740219,22,14.75,14,11.25,8,7.25
1,567a180372a293fe,3,0.998385,21,14.75,17,11.50,9,6.75
2,a30c99f77e48a7c2,7,0.470665,38,26.00,20,14.25,11,8.00
3,ab125199bd343017,11,0.832278,52,37.50,27,17.25,19,11.00
4,fae1178f794f297a,12,0.921411,82,66.00,19,16.00,18,14.00


In [270]:
df_new = df.merge(test_revised, how = 'left', on = 'id')
df_new.head()

,id,landmarks,landmark_id,confidence,0,1,2,3,4,5
0,000270c9100de789,4404 0.4304988384246826,4404,0.430499,25,13.50,17,9.75,10,6.50
1,0002b0fab5d3ccc4,11491 0.546776294708252,11491,0.546776,35,16.75,12,9.50,11,7.00
2,00055cf2bfb5594a,10184 0.597339928150177,10184,0.597340,26,16.75,12,9.25,12,8.50
3,0006aea5b6f4eaaa,6603 0.9498259425163269,6603,0.949826,71,45.25,34,24.00,16,12.75
4,0006bbfa00dd6c0f,9779 0.5040563344955444,9779,0.504056,53,21.75,31,13.00,15,8.50


In [271]:
df = df_new

In [272]:
threshold = 0

df_tmp = df.dropna(how = 'any')
df_tmp[['confidence']].describe()

,confidence
count,40283.000000
mean,0.673755
std,0.183091
min,0.400015
25%,0.509428
50%,0.650096
75%,0.833801
max,1.000000


In [40]:
df_tmp[['5']].describe()

,5
count,40283.000000
mean,9.121116
std,12.195929
min,0.000000
25%,5.000000
50%,7.500000
75%,11.250000
max,200.000000


In [41]:
pd.unique(df_tmp['5'])

array([  6.5       ,   7.        ,   8.5       ,  12.75      ,
        11.        ,  13.5       ,  19.25      ,   4.        ,
         4.5       ,   6.        ,  14.75      ,   9.5       ,
         3.25      ,  14.33333333,   9.        ,  13.75      ,
         6.75      ,  14.        ,   7.5       ,   8.25      ,
        14.25      ,   2.5       ,   0.5       , 200.        ,
         5.        ,  10.75      ,  12.25      ,   1.5       ,
        17.        ,   8.        ,   4.25      ,  19.5       ,
         5.25      ,   5.75      ,   6.25      ,   4.75      ,
         1.25      ,  16.25      ,   1.75      ,  13.        ,
        10.        ,   2.25      ,  12.5       ,   2.75      ,
         7.25      ,  11.5       ,  16.75      ,  12.        ,
         2.        ,  15.        ,   8.75      ,   9.75      ,
         9.66666667,  10.5       ,  11.25      ,  24.        ,
        16.        ,  20.        ,   3.        ,  22.5       ,
         7.75      ,   0.75      ,  14.5       ,  18.25

In [273]:
threshold_2 = 24.25

df_tmp = df_tmp[df_tmp['confidence'] > threshold]
df_tmp = df_tmp[np.logical_or(df_tmp['5'] == 22.25, df_tmp['5'] > threshold_2)]
df_tmp = df_tmp.drop(['landmark_id', 'confidence', '0', '1', '2', '3', '4', '5'], axis = 1)
print(df_tmp.shape)

(477, 3)


In [234]:
df_final = df_sample.merge(df_tmp, how = 'left', on = 'id')
df_final.to_csv('submission_118_5_22.25&24.25.csv', index = False)
df_final.shape

(117703, 2)